<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/CaptumLongformerSequenceClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import dependencies

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [ ]:
pip install transformers --quiet

     |████████████████████████████████| 4.4 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 51.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.6 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 


In [ ]:
pip install captum --quiet

     |████████████████████████████████| 1.4 MB 5.2 MB/s 


In [ ]:
pip install datasets --quiet

     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 55.7 MB/s 
     |████████████████████████████████| 1.1 MB 42.0 MB/s 
     |████████████████████████████████| 140 kB 11.2 MB/s 
     |████████████████████████████████| 127 kB 59.2 MB/s 
     |████████████████████████████████| 271 kB 42.1 MB/s 
     |████████████████████████████████| 144 kB 59.8 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Import model

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer, LongformerConfig
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'danielhou13/longformer-finetuned_papers_v2'
#model_path = 'danielhou13/longformer-finetuned-new-cogs402'

# load model
model = LongformerForSequenceClassification.from_pretrained(model_path, num_labels = 2)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/567M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Create functions that give us the input ids and the position ids for the text we want to examine

In [ ]:
def predict(inputs, position_ids=None):
    output = model(inputs,
                   position_ids=position_ids)
    return output.logits

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
max_length = 2048
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):

    text_ids = tokenizer.encode(text, truncation = True, add_special_tokens=False, max_length = max_length)
    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids


Import dataset and take one example from it for testing purposes

In [ ]:
from datasets import load_dataset
import numpy as np
cogs402_ds = load_dataset("danielhou13/cogs402dataset")["test"]

Downloading:   0%|          | 0.00/752 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402dataset-cc784554b797f843


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/danielhou13___parquet/danielhou13--cogs402dataset-cc784554b797f843/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# cogs402_ds2 = load_dataset('hyperpartisan_news_detection', 'bypublisher')['validation']
# val_size = 5000
# val_indices = np.random.randint(0, len(cogs402_ds2), val_size)
# val_ds = cogs402_ds2.select(val_indices)
# labels2 = map(int, val_ds['hyperpartisan'])
# labels2 = list(labels2)
# val_ds = val_ds.add_column("labels", labels2)

In [ ]:
testval = 923
text = cogs402_ds['text'][testval]
label = cogs402_ds['labels'][testval]
print(label)

0


In [ ]:
# testval = 923
# text = cogs402_ds2['text'][testval]
# label = cogs402_ds2['labels'][testval]
# print(label)

In [ ]:
#set 1 if we are dealing with a positive class, and 0 if dealing with negative class
def custom_forward(inputs, position_ids=None):
    preds = predict(inputs,
                   position_ids=position_ids
                   )
    return torch.softmax(preds, dim = 1)

Perform Layer Integrated Gradients using the longformer's embeddings

In [ ]:
lig = LayerIntegratedGradients(custom_forward, model.longformer.embeddings)

In [ ]:
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
# attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [ ]:
attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    target=label,
                                    n_steps=250,
                                    internal_batch_size = 2)

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.norm(attributions)
    return attributions

In [ ]:
attributions_sum = summarize_attributions(attributions)

In [ ]:
print(len(all_tokens))

2050


In [ ]:
score = predict(input_ids)

Visualize the attributions for the words

In [ ]:
# storing couple samples in an array for visualization purposes
score_vis = viz.VisualizationDataRecord(
                        attributions_sum,
                        torch.softmax(score, dim = 1).max(),
                        torch.argmax(torch.softmax(score, dim = 1)[label]),
                        label,
                        text,
                        attributions_sum.sum(),       
                        all_tokens,
                        delta)

print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])

 Visualization For Score 


See which words had the strongest (most positive and most negative) attributions

In [ ]:
def get_topk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [ ]:
def get_botk_attributed_tokens(attrs, k=20):
    values, indices = torch.topk(attrs, k, largest=False)
    top_tokens = [all_tokens[idx] for idx in indices]
    return top_tokens, values, indices

In [ ]:
import pandas as pd
top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum)
bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum)

df_high = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(top_words_start, top_word_ind_start, top_words_val_start)]})

df_low = pd.DataFrame({'Word(Index), Attribution': ["{} ({}), {}".format(word, pos, round(val.item(),2)) for word, pos, val in zip(bot_words_start, bot_word_ind_start, bot_words_val_start)]})
# df_start.style.apply(['cell_ids: False'])

# ['{}({})'.format(token, str(i)) for i, token in enumerate(all_tokens)]

In [ ]:
df_high

,"Word(Index), Attribution"
0,". (1781), 0.4"
1,". (1301), 0.18"
2,". (115), 0.12"
3,". (234), 0.1"
4,". (1040), 0.08"
5,". (278), 0.08"
6,". (392), 0.07"
7,"Ġperformance (149), 0.06"
8,") (251), 0.05"
9,". (1419), 0.05"


In [ ]:
df_low

,"Word(Index), Attribution"
0,". (150), -0.8"
1,"Ġconstants (1536), -0.2"
2,". (185), -0.13"
3,", (1036), -0.12"
4,". (64), -0.09"
5,"( (512), -0.08"
6,". (390), -0.08"
7,". (1579), -0.06"
8,"Ġthat (252), -0.05"
9,"Ġoptimizations (1024), -0.05"


In [ ]:
d = {"tokens":all_tokens, "attribution":attributions_sum[len(all_tokens)].cpu()}

In [ ]:
print(len(all_tokens), len(attributions_sum))

2050 2560


Add all the duplicate tokens together to get the total for each token

In [ ]:
df_attrib = pd.DataFrame(d)
aggregation_functions = {'attribution': 'sum'}
df_new = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_functions)

In [ ]:
highest_attrib_tokens = df_new.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens[:10]

,attribution
tokens,
Ġto,0.202729
.,0.081005
Ġand,0.072803
Ġperformance,0.056295
Ġprograms,0.055392
Ġprogramming,0.055188
Ġprogram,0.054239
Ġby,0.052643
Ġlanguage,0.051812


In [ ]:
lowest_attrib_tokens = df_new.sort_values(by=['attribution'])
lowest_attrib_tokens[:10]

,attribution
tokens,
Ġconstants,-0.194931
Ġthe,-0.083060
Ġoptimizations,-0.049456
Ġ,-0.049209
",",-0.049196
Ġthat,-0.045101
ĠTo,-0.043678
(,-0.040371
Ġproposed,-0.028464


In [ ]:
print(len(all_tokens), attributions_sum[:len(all_tokens)].shape)

2050 torch.Size([2050])


In [ ]:
from tqdm import tqdm
aggregate_attrib_zero = []
aggregate_attrib_ones = []

for i in tqdm(range(len(cogs402_ds))):
  text = cogs402_ds[i]['text']
  label = cogs402_ds[i]['labels']
  input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
  position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
  # attention_mask = construct_attention_mask(input_ids)

  indices = input_ids[0].detach().tolist()
  all_tokens = tokenizer.convert_ids_to_tokens(indices)

  attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines=ref_input_ids,
                                    return_convergence_delta=True,
                                    target=label,
                                    n_steps=15,
                                    internal_batch_size = 2)
  attributions_sum = summarize_attributions(attributions)

  top_words_start, top_words_val_start, top_word_ind_start = get_topk_attributed_tokens(attributions_sum)
  bot_words_start, bot_words_val_start, bot_word_ind_start = get_botk_attributed_tokens(attributions_sum)

  d = {"tokens":all_tokens, "attribution":attributions_sum[:len(all_tokens)].cpu()}  
  aggregation_function = {'attribution': 'sum'}
  df_attrib = pd.DataFrame(d)
  df_attrib = df_attrib.groupby(df_attrib['tokens']).aggregate(aggregation_function)

  if label == 0:
    aggregate_attrib_zero.append(df_attrib)
  else:
    aggregate_attrib_ones.append(df_attrib)

100%|██████████| 1179/1179 [3:33:20<00:00, 10.86s/it]


In [ ]:
df_attrib_zero = pd.concat(aggregate_attrib_zero)
df_attrib_zero = df_attrib_zero.reset_index(level=0)
df_attrib_zero = df_attrib_zero.groupby(df_attrib_zero['tokens']).aggregate(aggregation_function)
df_attrib_zero['attribution'] = df_attrib_zero['attribution'].div(len(aggregate_attrib_zero))
highest_attrib_tokens_all = df_attrib_zero.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens_all[:10]

,attribution
tokens,
Ġto,0.137865
Ġ.,0.126639
Ġprogram,0.042907
Ġand,0.039635
1,0.030574
Ġprogramming,0.030277
Ġlanguage,0.027670
Ġfor,0.025824
Ġof,0.025143


In [ ]:
df_attrib_ones = pd.concat(aggregate_attrib_ones)
df_attrib_ones = df_attrib_ones.reset_index(level=0)
df_attrib_ones = df_attrib_ones.groupby(df_attrib_ones['tokens']).aggregate(aggregation_function)
df_attrib_ones['attribution'] = df_attrib_ones['attribution'].div(len(aggregate_attrib_ones))
highest_attrib_tokens_all2 = df_attrib_ones.sort_values(by=['attribution'], ascending=False)
highest_attrib_tokens_all2[:10]

,attribution
tokens,
Ġthe,0.282937
.,0.282339
Ġlearning,0.171072
Ġ,0.067647
Ġneural,0.059037
Ġtraining,0.041787
Ġa,0.036133
AI,0.026528
-,0.022634


In [ ]:
highest_attrib_tokens_all.to_csv('data/longformer_emb_zeros_papers.csv')  
highest_attrib_tokens_all2.to_csv('data/longformer_emb_ones_papers.csv')  